In [1]:
import pandas as pd
import numpy as np
from glob import glob

In [6]:
df_list = glob('ensemble/*')

for index, df in enumerate(df_list):
    if index == 0:
        merged_df = pd.read_csv(df)
    else:
        df_temp = pd.read_csv(df)
        merged_df = pd.merge(merged_df, df_temp, on='img_id', suffixes=(f'_{index}', f'_{index+1}'))



In [7]:
merged_df

,img_id,mask_rle_1,mask_rle_2,mask_rle_3,mask_rle_4,mask_rle
0,TEST_00000,19780 6 20002 10 20226 10 20255 9 20450 10 204...,19780 1 20002 10 20226 11 20255 10 20450 11 20...,20002 9 20033 5 20226 10 20256 8 20450 11 2048...,19779 2 20002 11 20226 11 20256 8 20450 11 204...,20002 10 20226 11 20256 8 20450 11 20479 9 206...
1,TEST_00001,35614 3 35837 4 36059 6 36282 7 36504 9 36727 ...,35839 2 36061 4 36283 6 36505 8 36728 9 36950 ...,35615 2 35837 4 36060 5 36283 6 36505 8 36729 ...,36062 3 36284 5 36506 7 36728 9 36950 11 37172...,35838 3 36059 6 36282 7 36504 9 36727 10 36949...
2,TEST_00002,-1,-1,-1,-1,-1
3,TEST_00003,18 28 67 1 72 11 242 28 291 1 296 11 466 28 52...,24 21 73 12 246 23 297 11 467 27 522 8 689 29 ...,21 24 74 12 245 24 298 12 467 27 522 10 689 29...,23 22 72 13 246 23 297 10 467 27 522 8 690 28 ...,21 24 73 12 245 24 297 12 467 26 521 10 690 28...
4,TEST_00004,16894 17 17115 45 17338 46 17562 46 17786 47 1...,16897 2 16900 12 17114 23 17154 5 17338 24 173...,16892 20 17114 24 17148 11 17338 46 17562 46 1...,16904 8 17114 23 17151 9 17338 46 17562 46 177...,16686 1 16892 20 16931 4 17114 24 17149 11 173...
...,...,...,...,...,...,...
60635,TEST_60635,-1,-1,-1,-1,-1
60636,TEST_60636,29754 6 29974 12 29989 5 30196 14 30212 6 3041...,30213 6 30429 5 30436 8 30648 20 30871 22 3109...,29767 1 29978 3 29987 9 30198 9 30208 14 30420...,27976 4 28190 15 28410 20 28631 24 28852 27 29...,30202 2 30422 8 30644 11 30868 12 31091 14 313...
60637,TEST_60637,88 19 312 19 536 19 761 17 985 17 1209 17 1433...,57 15 89 19 280 16 313 19 504 16 537 18 728 17...,89 14 313 14 537 15 761 15 985 16 1209 16 1403...,57 13 86 25 281 13 310 25 505 14 534 25 729 14...,90 16 314 16 538 16 762 16 986 16 1210 16 1435...
60638,TEST_60638,15158 5 15376 12 15595 18 15818 20 16042 20 16...,-1,15152 7 15374 12 15598 12 15822 13 16046 13 16...,-1,-1


In [4]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()

    if pixels.sum() == 0:
        return -1
    
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

In [8]:
def hard_vote(row):
    masks = row[['mask_rle_1', 'mask_rle_2', 'mask_rle_3', 'mask_rle_4', 'mask_rle']].values
    mask_array = np.zeros((224, 224))
    for mask in masks:
        if isinstance(mask, str):
            decoded_mask = rle_decode(mask, (224, 224))  # Replace with your RLE decoding logic
            mask_array += decoded_mask

    mask_array = (mask_array >= 3).astype(int)  # Adjust the threshold for voting agreement
    
    return rle_encode(mask_array)  # Replace with your RLE encoding logic

merged_df['final_mask_rle'] = merged_df.apply(hard_vote, axis=1)

In [9]:
merged_df = merged_df[['img_id', 'final_mask_rle']]
merged_df.rename(columns = {'final_mask_rle':'mask_rle'}, inplace = True)
merged_df

/tmp/ipykernel_225196/1623989734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.rename(columns = {'final_mask_rle':'mask_rle'}, inplace = True)


,img_id,mask_rle
0,TEST_00000,19780 1 20002 10 20226 11 20256 8 20450 11 204...
1,TEST_00001,35838 3 36060 5 36283 6 36505 8 36728 9 36950 ...
2,TEST_00002,
3,TEST_00003,21 24 73 12 245 24 297 11 467 27 522 8 690 28 ...
4,TEST_00004,16894 18 17114 24 17149 11 17338 46 17562 46 1...
...,...,...
60635,TEST_60635,
60636,TEST_60636,29978 3 29989 5 30198 9 30208 2 30212 7 30420 ...
60637,TEST_60637,89 18 313 18 537 18 761 17 985 17 1209 17 1403...
60638,TEST_60638,


In [10]:
merged_df.to_csv('ensemble_result.csv', index=False)

In [13]:
merged_df = pd.read_csv('ensemble_result.csv')
merged_df

,img_id,mask_rle
0,TEST_00000,19780 1 20002 10 20226 11 20256 8 20450 11 204...
1,TEST_00001,35838 3 36060 5 36283 6 36505 8 36728 9 36950 ...
2,TEST_00002,NaN
3,TEST_00003,21 24 73 12 245 24 297 11 467 27 522 8 690 28 ...
4,TEST_00004,16894 18 17114 24 17149 11 17338 46 17562 46 1...
...,...,...
60635,TEST_60635,NaN
60636,TEST_60636,29978 3 29989 5 30198 9 30208 2 30212 7 30420 ...
60637,TEST_60637,89 18 313 18 537 18 761 17 985 17 1209 17 1403...
60638,TEST_60638,NaN


In [15]:
merged_df.fillna(-1, inplace=True)
merged_df

,img_id,mask_rle
0,TEST_00000,19780 1 20002 10 20226 11 20256 8 20450 11 204...
1,TEST_00001,35838 3 36060 5 36283 6 36505 8 36728 9 36950 ...
2,TEST_00002,-1
3,TEST_00003,21 24 73 12 245 24 297 11 467 27 522 8 690 28 ...
4,TEST_00004,16894 18 17114 24 17149 11 17338 46 17562 46 1...
...,...,...
60635,TEST_60635,-1
60636,TEST_60636,29978 3 29989 5 30198 9 30208 2 30212 7 30420 ...
60637,TEST_60637,89 18 313 18 537 18 761 17 985 17 1209 17 1403...
60638,TEST_60638,-1


In [16]:
merged_df.to_csv('ensemble_result.csv', index=False)